In [ ]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob
from copy import copy
from tqdm.notebook import tqdm
import warnings
import staintools

# 경고 메시지를 무시하도록 설정s
warnings.filterwarnings("ignore")

In [ ]:
protein_expression_list=glob('../../data/virtual_stain/image_surf/svs/**/*.svs')
protein_expression_save_list=[f.replace('svs/', 'thumb/') for f in protein_expression_list]
df = pd.DataFrame(columns=['slide','stain','ratio'])
def size_ratio(scene,img_size):
    width = scene.rect[2]
    height = scene.rect[3]
    ratio=0
    inverse_ratio=0
    img_width=0
    img_height=0
    if width>height:
        ratio=img_size/width
        inverse_ratio=width/img_size
        img_width=img_size
        img_height=height*ratio
    else:
        ratio=img_size/height
        inverse_ratio=height/img_size
        img_height=img_size
        img_width=width*ratio
        
    return int(img_width),int(img_height),inverse_ratio

    

In [ ]:
for i in tqdm(range(len(protein_expression_list))):
    slide = slideio.open_slide(protein_expression_list[i], "SVS")
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    slide_name=os.path.splitext(os.path.basename(protein_expression_list[i]))[0]
    stain_name=os.path.basename(os.path.dirname(protein_expression_list[i])) 
    sample=[slide_name,stain_name,ratio]       
    df.loc[i]=sample
    cv2.imwrite(os.path.splitext(protein_expression_save_list[i])[0]+'.tiff', image)

In [ ]:
for i in tqdm(range(len(protein_expression_list))):
    slide = slideio.open_slide(protein_expression_list[i], "SVS")
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    slide_name=os.path.splitext(os.path.basename(protein_expression_list[i]))[0]
    stain_name=os.path.basename(os.path.dirname(protein_expression_list[i])) 
    sample=[slide_name,stain_name,ratio]       
    df.loc[i]=sample

In [7]:
df.to_csv('../../data/virtual_stain/image_surf/data_frame.csv',index=False)